# 3 Classificação (cont.)

## 3.3) O aumento da quantidade professores nas instituições de ensino, de estudantes matriculados na educação primária e no número de professores por aluno inflencia na taxa de mortalidade infantil?

### Instalando e importando as bibliotecas utlizadas nesse notebook.

In [1]:
!pip3 install pandas
!pip3 install numpy
!pip3 install sklearn

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
import numpy as np

## 3.2.1) Carga e transformação da massa bruta de todos os indicadores e países.

#### Nessa etapa é carregado o dataset "massa_bruta_pais_por_indicadores.csv" (resultado do notebook "2_extracao_transformacao_massa_indicadores_todos_paises") para usar os dados de países e seus respectivos indicadores.

In [3]:
massa_bruta_pais_por_indicadores = pd.read_csv('../data/massa_bruta_pais_por_indicadores.csv')

#### Após essa carga, são selecionados as colunas dos anos de 1998 a 2013 (esse intervalo foi identificado no notebook "1_extracao_faixa_temporal_melhor_distribuicao_valores") e as linhas dos indicadores de mortalidade infantil e educação..

- 'SH.DYN.MORT': taxa de mortalidade de crianças com até 5 anos (por 1.000 nascidos vivos).
- 'SE.PRM.TCHR': quantidade de professores na educação primária.
- 'SE.PRM.ENRL.TC.ZS': proporção de alunos por professor na educação primária.
- 'SE.PRM.ENRL': alunos inscritos no ensino primário.

In [4]:
anos = ['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', 
        '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']

indicadores = ['SH.DYN.MORT', 'SE.PRM.TCHR', 'SE.PRM.ENRL.TC.ZS', 'SE.PRM.ENRL']

indicador_mortalidade_infantil = ['SH.DYN.MORT']

indicadores_educacao_primaria = ['SE.PRM.TCHR', 'SE.PRM.ENRL.TC.ZS', 'SE.PRM.ENRL']

#### Aqui há extração dos nomes dos países (extraídos do dataset "massa_bruta_pais_por_indicadores.csv", através da filtragem de valores únicos). Em seguida, os dados desse dataset são filtrados pelos indicadores (especificados na célula acima, "indicadores").

In [5]:
paises = massa_bruta_pais_por_indicadores['Country Name'].unique()

df_massa_bruta_por_indicador = massa_bruta_pais_por_indicadores.loc[massa_bruta_pais_por_indicadores['Indicator Code'].isin(indicadores)]

colunas_transformacao = ['Pais', 'Indicador', '1998', '1999', '2000', '2001', '2002', '2003', 
                         '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012']


#### O passo abaixo processará os dados filtrados pelos indicadores de mortalidade infantil e educação primária:
- Trocar o valor das linhas do indicador de mortalidade por zero ou um (um quando houver aumento de um ano para o outro e zero não);
- Preencher o dataframe "df_resultado_transformacao" com o resultado disso;
- Substituir os valores em branco por zero;
- Ordernar "df_resultado_transformacao" pelo nome do país; e
- Ajustar os índices de "df_resultado_transformacao".

#### Os dados no dataframe "df_resultado_transformacao" estarão agrupados por país e indicadores.
#### Os valores de zero ou um nas linhas referentes ao indicador de mortalidade mostram se, em cada ano, houve um aumento ou não da taxa de mortalidade infantil.

In [6]:
df_resultado_transformacao = pd.DataFrame(columns=colunas_transformacao)

for pais in paises:
    df_pais = df_massa_bruta_por_indicador.loc[df_massa_bruta_por_indicador['Country Name'] == pais]

    df_1998_a_2013 = df_pais.iloc[:, 2:18]
    df_1998_a_2013.index = range(df_1998_a_2013.shape[0])
    df_1998_a_2013 = df_1998_a_2013.fillna(0)
   
    indicador = df_pais['Indicator Code']
    indicador.index = range(indicador.shape[0])
    
    for index, indicador in indicador.items():
        
        if indicador in indicador_mortalidade_infantil:
            valor_1998_1999 = 1 if df_1998_a_2013['1999'].iloc[index] - df_1998_a_2013['1998'].iloc[index] > 0 else 0
            valor_1999_2000 = 1 if df_1998_a_2013['2000'].iloc[index] - df_1998_a_2013['1999'].iloc[index] > 0 else 0
            valor_2000_2001 = 1 if df_1998_a_2013['2001'].iloc[index] - df_1998_a_2013['2000'].iloc[index] > 0 else 0
            valor_2001_2002 = 1 if df_1998_a_2013['2002'].iloc[index] - df_1998_a_2013['2001'].iloc[index] > 0 else 0
            valor_2002_2003 = 1 if df_1998_a_2013['2003'].iloc[index] - df_1998_a_2013['2002'].iloc[index] > 0 else 0
            valor_2003_2004 = 1 if df_1998_a_2013['2004'].iloc[index] - df_1998_a_2013['2003'].iloc[index] > 0 else 0
            valor_2004_2005 = 1 if df_1998_a_2013['2005'].iloc[index] - df_1998_a_2013['2004'].iloc[index] > 0 else 0
            valor_2005_2006 = 1 if df_1998_a_2013['2006'].iloc[index] - df_1998_a_2013['2005'].iloc[index] > 0 else 0
            valor_2006_2007 = 1 if df_1998_a_2013['2007'].iloc[index] - df_1998_a_2013['2006'].iloc[index] > 0 else 0
            valor_2007_2008 = 1 if df_1998_a_2013['2008'].iloc[index] - df_1998_a_2013['2007'].iloc[index] > 0 else 0
            valor_2008_2009 = 1 if df_1998_a_2013['2009'].iloc[index] - df_1998_a_2013['2008'].iloc[index] > 0 else 0
            valor_2009_2010 = 1 if df_1998_a_2013['2010'].iloc[index] - df_1998_a_2013['2009'].iloc[index] > 0 else 0
            valor_2010_2011 = 1 if df_1998_a_2013['2011'].iloc[index] - df_1998_a_2013['2010'].iloc[index] > 0 else 0
            valor_2011_2012 = 1 if df_1998_a_2013['2012'].iloc[index] - df_1998_a_2013['2011'].iloc[index] > 0 else 0
            valor_2012_2013 = 1 if df_1998_a_2013['2013'].iloc[index] - df_1998_a_2013['2012'].iloc[index] > 0 else 0

            valores = [pais, indicador, valor_1998_1999, valor_1999_2000, valor_2000_2001, valor_2001_2002, 
                       valor_2002_2003, valor_2003_2004, valor_2004_2005, valor_2005_2006, valor_2006_2007, 
                       valor_2007_2008, valor_2008_2009, valor_2009_2010, valor_2010_2011, valor_2011_2012,
                       valor_2012_2013]

            df_auxiliar = pd.DataFrame([valores], columns=colunas_transformacao)
            df_auxiliar.index = range(df_auxiliar.shape[0])
            df_resultado_transformacao = pd.concat([df_auxiliar, df_resultado_transformacao])            

        elif indicador in indicadores_educacao_primaria:
            valores = [pais, indicador]
            valores.extend(df_1998_a_2013.drop(columns=['2013']).iloc[index].values.tolist())
            df_auxiliar = pd.DataFrame([valores], columns=colunas_transformacao)
            df_auxiliar.index = range(df_auxiliar.shape[0])
            df_resultado_transformacao = pd.concat([df_auxiliar, df_resultado_transformacao])

            
df_resultado_transformacao = df_resultado_transformacao.sort_values(['Pais'], ascending=[True])
df_resultado_transformacao.index = range(df_resultado_transformacao.shape[0])
df_resultado_transformacao.head(10)

,Pais,Indicador,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
0,Afghanistan,SE.PRM.ENRL,1046338.0,875605.00,749360.00,773623.00,2667629.0,3781015.0,4430142.0,4318819.0,4669110.00,4718077.00,4974836.00,4945632.00,5279326.00,5291624.00,5767543.00
1,Afghanistan,SE.PRM.TCHR,32593.0,26385.00,0.00,0.00,0.0,0.0,0.0,0.0,110313.00,110312.00,113763.00,115454.00,118858.00,121593.00,129093.00
2,Afghanistan,SH.DYN.MORT,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Afghanistan,SE.PRM.ENRL.TC.ZS,32.1,33.19,0.00,0.00,0.0,0.0,0.0,0.0,42.33,42.77,43.73,42.84,44.42,43.52,44.68
4,Albania,SE.PRM.TCHR,0.0,0.00,12551.00,12607.00,0.0,11762.0,0.0,0.0,0.00,0.00,10277.00,11701.00,11409.00,10854.00,10605.00
5,Albania,SE.PRM.ENRL,299164.0,292070.00,283249.00,274233.00,263603.0,252829.0,240487.0,237975.0,222402.00,210125.00,199454.00,236102.00,224781.00,215660.00,206617.00
6,Albania,SH.DYN.MORT,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Albania,SE.PRM.ENRL.TC.ZS,0.0,0.00,22.57,21.75,0.0,21.5,0.0,0.0,0.00,0.00,19.41,20.18,19.70,19.87,19.48
8,Algeria,SH.DYN.MORT,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Algeria,SE.PRM.TCHR,0.0,169519.00,170562.00,169559.00,170039.0,167529.0,170031.0,171471.0,171402.00,170775.00,169701.00,141421.00,141994.00,144537.00,149036.00


#### Transforma todos os dados dos indicadores em colunas. Isso será útil para as etapas de análise e classificação. Também renomeia as colunas para facilitar o entendimento.

In [7]:
df_mortalidade_educacao_primaria = pd.DataFrame()

for indicador in indicadores:
    df_indicador = df_resultado_transformacao.loc[df_resultado_transformacao['Indicador'] == indicador]
    df_indicador.index = range(df_indicador.shape[0])
    df_periodos_indicador = df_indicador.drop(['Pais', 'Indicador'], axis=1)    
    df_periodos_indicador = df_periodos_indicador.stack()
    df_periodos_indicador.index = range(df_periodos_indicador.shape[0])
    df_mortalidade_educacao_primaria[indicador] = df_periodos_indicador.tolist()

    
nomes_colunas = {'SH.DYN.MORT': 'Mortalidade Infantil', 
                 'SE.PRM.TCHR': 'Qtd Professores Primario', 
                 'SE.PRM.ENRL.TC.ZS': 'Alunos por Professor Primario',
                 'SE.PRM.ENRL': 'Estudantes Primario'}

df_mortalidade_educacao_primaria = df_mortalidade_educacao_primaria.rename(columns = nomes_colunas)
df_mortalidade_educacao_primaria.head()

,Mortalidade Infantil,Qtd Professores Primario,Alunos por Professor Primario,Estudantes Primario
0,0.0,32593.0,32.10,1046338.0
1,0.0,26385.0,33.19,875605.0
2,0.0,0.0,0.00,749360.0
3,0.0,0.0,0.00,773623.0
4,0.0,0.0,0.00,2667629.0


#### Antes de começar a etapa de classificação, verifica-se se a coluna "Mortalidade Infantil" possui alta taxa de preenchimento ou boa distribuição de valores.

In [8]:
## Extrai os dados originais da coluna referente ao indicador de mortalidade infantil ##

df_analise_mortalidade = pd.DataFrame(columns=colunas_transformacao)

for pais in paises:
    df_pais = df_massa_bruta_por_indicador.loc[df_massa_bruta_por_indicador['Country Name'] == pais]

    df_1998_a_2013 = df_pais.iloc[:, 2:18]
    df_1998_a_2013.index = range(df_1998_a_2013.shape[0])
    df_1998_a_2013 = df_1998_a_2013.fillna(0)
   
    indicador = df_pais['Indicator Code']
    indicador.index = range(indicador.shape[0])
    
    for index, indicador in indicador.items():        
        if indicador in indicador_mortalidade_infantil:
            valores = [pais, indicador]
            valores.extend(df_1998_a_2013.drop(columns=['2013']).iloc[index].values.tolist())
            df_auxiliar = pd.DataFrame([valores], columns=colunas_transformacao)
            df_auxiliar.index = range(df_auxiliar.shape[0])
            df_analise_mortalidade = pd.concat([df_auxiliar, df_analise_mortalidade])
            
df_analise_mortalidade = df_analise_mortalidade.sort_values(['Pais'], ascending=[True])
df_analise_mortalidade.index = range(df_analise_mortalidade.shape[0])
## Fim da extração ##


# Agora os dados da coluna de mortalidade infantil de todos os países para os anos de 1998 a 2012 
# serão empilhados em uma única coluna.
df_1998_a_2013 = df_analise_mortalidade.iloc[:, 2:18]
df_1998_a_2013.index = range(df_1998_a_2013.shape[0])
df_periodos_indicador = df_1998_a_2013.stack()
df_periodos_indicador.index = range(df_periodos_indicador.shape[0])
df_analise_educacao_primaria = pd.DataFrame()
df_analise_educacao_primaria['Mortalidade Infantil'] = df_periodos_indicador.tolist()


# Os dados originais de mortalidade infantil são descritos e é contada a quantidade valores não preenchidos.
qtd_zeros = df_analise_educacao_primaria['Mortalidade Infantil'].isin(['0']).mean()*100
print("Quantidade de valores não preenchidos para a coluna de mortalidade infantil: %.2f%%" % qtd_zeros)

# Ao realizar uma descrição dos dados de mortalidade infantil percebe-se que há uma
# pequena incidência de campos com valor igual a zero.

Quantidade de valores não preenchidos para a coluna de mortalidade infantil: 11.52%


#### A função "classificar" objetiva analisar o resultado de um modelo de classificação. Além disso, imprime interpretações das previsões.

In [9]:
def classificar(modelo, treino_x, teste_x, treino_y, teste_y, nome_modelo):
    modelo.fit(treino_x, treino_y)
    print("*********  " + nome_modelo + "  *********")
    print("Acurácia treinamento: %.2f%%" % (modelo.score(treino_x, treino_y)*100))
    previsoes = modelo.predict(teste_x)
    print("Acurácia previsão: %.2f%%" % (accuracy_score(teste_y, previsoes)*100))
    print("")
    print("<< Matriz de confusão >>")
    matriz_confusao = confusion_matrix(teste_y, previsoes)
    print(pd.crosstab(teste_y, previsoes, rownames=['Real'], colnames=['Predito'], margins=True))
    print("")
    print("<< Relatório de classificação >>")
    print(classification_report(teste_y, previsoes))

#### Aplicação do "Dummy Classifier" e sua respectiva acurácia.

In [10]:
x = df_mortalidade_educacao_primaria[['Qtd Professores Primario', 'Alunos por Professor Primario', 'Estudantes Primario']]
y = df_mortalidade_educacao_primaria['Mortalidade Infantil']


#Define a ordem dos números aleatórios e tetira a aleatoriedade da separação dos dados de treino e teste.
SEED = 5
np.random.seed(SEED)

treino_x_bruto_dm, teste_x_bruto_dm, treino_y_dm, teste_y_dm = train_test_split(x, y, random_state = SEED, test_size=0.25, stratify=y)
#O parâmetro "stratify=y" orienta o algoritmo a estratificar (separar proporcionalmente) os dados de acordo com o "y".

classificar(DummyClassifier(strategy='stratified'), treino_x_bruto_dm, teste_x_bruto_dm, treino_y_dm, teste_y_dm, "Dummy")

*********  Dummy  *********
Acurácia treinamento: 94.18%
Acurácia previsão: 94.47%

<< Matriz de confusão >>
Predito  0.0  1.0  All
Real                  
0.0      767   21  788
1.0       24    2   26
All      791   23  814

<< Relatório de classificação >>
              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97       788
         1.0       0.09      0.08      0.08        26

    accuracy                           0.94       814
   macro avg       0.53      0.53      0.53       814
weighted avg       0.94      0.94      0.94       814



#### Aplicação do "SVC" e sua acurácia.

In [11]:
# Utilização do modelo "SVC"

x = df_mortalidade_educacao_primaria[['Qtd Professores Primario', 'Alunos por Professor Primario', 'Estudantes Primario']]
y = df_mortalidade_educacao_primaria['Mortalidade Infantil']

#Define a ordem dos números aleatórios e retira a aleatoriedade da separação dos dados de treino e teste.
SEED = 5
np.random.seed(SEED)

treino_x_bruto_svc, teste_x_bruto_svc, treino_y_svc, teste_y_svc = train_test_split(x, y, random_state = SEED, test_size=0.25, stratify=y)
#O parâmetro "stratify=y" orienta o algoritmo a estratificar (separar proporcionalmente) os dados de acordo com o "y".

#Reescalando os dados para uma faixa similar a todos.
scaler = StandardScaler()
scaler.fit(treino_x_bruto_svc)
treino_x = scaler.transform(treino_x_bruto_svc)
teste_x = scaler.transform(teste_x_bruto_svc)

classificar(SVC(), treino_x_bruto_svc, teste_x_bruto_svc, treino_y_svc, teste_y_svc, "SVC")

*********  SVC  *********
Acurácia treinamento: 96.85%
Acurácia previsão: 96.81%

<< Matriz de confusão >>
Predito  0.0  All
Real             
0.0      788  788
1.0       26   26
All      814  814

<< Relatório de classificação >>
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98       788
         1.0       0.00      0.00      0.00        26

    accuracy                           0.97       814
   macro avg       0.48      0.50      0.49       814
weighted avg       0.94      0.97      0.95       814



/home/joca/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Aplicação do "Gaussian Naive Bayes" e sua respectiva acurácia.

In [12]:
x = df_mortalidade_educacao_primaria[['Qtd Professores Primario', 'Alunos por Professor Primario', 'Estudantes Primario']]
y = df_mortalidade_educacao_primaria['Mortalidade Infantil']

#Define a ordem dos números aleatórios e tetira a aleatoriedade da separação dos dados de treino e teste.
SEED = 20
np.random.seed(SEED)

treino_x_bruto_nb, teste_x_bruto_nb, treino_y_nb, teste_y_nb = train_test_split(x, y, random_state = SEED, test_size=0.25, stratify=y)
#O parâmetro "stratify=y" orienta o algoritmo a estratificar (separar proporcionalmente) os dados de acordo com o "y".

classificar(GaussianNB(), treino_x_bruto_nb, teste_x_bruto_nb, treino_y_nb, teste_y_nb, "Gaussian Naive Bayes")

*********  Gaussian Naive Bayes  *********
Acurácia treinamento: 96.85%
Acurácia previsão: 96.81%

<< Matriz de confusão >>
Predito  0.0  All
Real             
0.0      788  788
1.0       26   26
All      814  814

<< Relatório de classificação >>
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98       788
         1.0       0.00      0.00      0.00        26

    accuracy                           0.97       814
   macro avg       0.48      0.50      0.49       814
weighted avg       0.94      0.97      0.95       814



/home/joca/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Aplicação do "Decision Tree Classifier" e sua respectiva acurácia. Além disso, é exibido um gráfico para a análise das decisões tomadas pelo algoritmo.

In [13]:
x = df_mortalidade_educacao_primaria[['Qtd Professores Primario', 'Alunos por Professor Primario', 'Estudantes Primario']]
y = df_mortalidade_educacao_primaria['Mortalidade Infantil']

#Define a ordem dos números aleatórios e tetira a aleatoriedade da separação dos dados de treino e teste.
SEED = 20
np.random.seed(SEED)

treino_x_bruto_dtc, teste_x_bruto_dtc, treino_y_dtc, teste_y_dtc = train_test_split(x, y, random_state = SEED, test_size=0.25, stratify=y)
#O parâmetro "stratify=y" orienta o algoritmo a estratificar (separar proporcionalmente) os dados de acordo com o "y".

modelo = DecisionTreeClassifier(max_depth=5)

classificar(DecisionTreeClassifier(max_depth=5), treino_x_bruto_dtc, teste_x_bruto_dtc, treino_y_dtc, teste_y_dtc, "DecisionTreeClassifier")

modelo.fit(treino_x_bruto_dtc, treino_y_dtc)
previsoes = modelo.predict(teste_x_bruto_dtc)
acuracia = accuracy_score(teste_y_dtc, previsoes)
features = x.columns
dados_dot = export_graphviz(modelo, feature_names=features, rounded=True, filled=True, class_names=['não', 'sim'])
imagem = graphviz.Source(dados_dot)
imagem

*********  DecisionTreeClassifier  *********
Acurácia treinamento: 97.54%
Acurácia previsão: 96.93%

<< Matriz de confusão >>
Predito  0.0  1.0  All
Real                  
0.0      783    5  788
1.0       20    6   26
All      803   11  814

<< Relatório de classificação >>
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98       788
         1.0       0.55      0.23      0.32        26

    accuracy                           0.97       814
   macro avg       0.76      0.61      0.65       814
weighted avg       0.96      0.97      0.96       814



#### Essa lógica calcula qual a média de zeros do indicador de mortalidade infantil "SH.DYN.MORT" (coluna a ser usada na classificação binária).

In [14]:
df = df_massa_bruta_por_indicador.loc[df_massa_bruta_por_indicador['Indicator Code'] == 'SH.DYN.MORT']
df.index = range(df.shape[0])
df = df.iloc[:, 2:18]
print("A média de valores não preenchidos (que receberão o valor zero) é de %.2f%%" % (df.isna().mean().mean() * 100))

A média de valores não preenchidos (que receberão o valor zero) é de 11.52%
